<a href="https://colab.research.google.com/github/isaacgrove/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Assignment_DS_121_Statistics_Probability_IsaacG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [7]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-21 16:10:17--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2020-01-21 16:10:17 (129 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [16]:
column_headers = ['party', 'handicapped-infants', 'water-project', 
'budget', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 
'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration', 
'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free', 
'south-africa']

df = pd.read_csv('house-votes-84.data', 
                 header=None, 
                 names=column_headers, 
                 na_values='?')
                 
df = df.replace({'y': 1, 'n': 0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# You can tell whether the support is "greater than" or "less than" based on the sign of the t statistic.

In [18]:
rep = df[df['party']=='republican']
rep.head()


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
|

In [20]:
dem = df[df['party']=='democrat']
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [24]:
for column in rep:
  if column != 'party':
    print(rep[column].mean())

0.18787878787878787
0.5067567567567568
0.13414634146341464
0.9878787878787879
0.9515151515151515
0.8975903614457831
0.24074074074074073
0.15286624203821655
0.11515151515151516
0.5575757575757576
0.1320754716981132
0.8709677419354839
0.8607594936708861
0.9813664596273292
0.08974358974358974
0.6575342465753424


In [25]:
for column in dem:
  if column != 'party':
    print(dem[column].mean())

0.6046511627906976
0.502092050209205
0.8884615384615384
0.05405405405405406
0.21568627450980393
0.47674418604651164
0.7722007722007722
0.8288973384030418
0.7580645161290323
0.4714828897338403
0.5058823529411764
0.14457831325301204
0.2896825396825397
0.35019455252918286
0.6374501992031872
0.9351351351351351


In [26]:
# try physician-fee-freeze for rep > dem
# Null hypothesis: no difference in support between the two parties.
print(rep['physician-fee-freeze'].mean())
print(dem['physician-fee-freeze'].mean())

0.9878787878787879
0.05405405405405406


In [29]:
ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy='omit')

# Due to a p-value of way (way) less than 0.01, we reject the null hypothesis
# and conclude that there is a significant difference in support between
# Republicans and Democrats. Republicans support this bill more, as evidenced 
# by the positive t-stat and visually examining the mean support levels above.
# Also note that a one-sided t-test would be easier to reject the null, so we
# have effectively given ourselves a 99.5% confidence level that republican
# support is greater than democrat support.

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

In [0]:
#

In [31]:
# try anti-satellite-ban for dem > rep
# Null hypothesis: no difference in support between the two parties.
print(rep['anti-satellite-ban'].mean())
print(dem['anti-satellite-ban'].mean())

0.24074074074074073
0.7722007722007722


In [30]:
ttest_ind(rep['anti-satellite-ban'], dem['anti-satellite-ban'], nan_policy='omit')

# Due to p-value < 0.01 and a negative t-stat, 
# we reject the null hypothesis that there is no difference
# in support and conclude that democrat support is higher than republican support.
# 

Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)

In [0]:
#

In [32]:
## try water-project for dem = rep
# Null hypothesis: no difference in support between the two parties.
print(rep['water-project'].mean())
print(dem['water-project'].mean())

0.5067567567567568
0.502092050209205


In [33]:
ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')

# Due to p-value of ~0.92 and a t-stat of ~0.089, 
# we fail to reject the null hypothesis that there is no difference
# in support.

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)